In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Data EDA

In [ ]:
# package load
import matplotlib.pyplot as plt
import seaborn as sns

## 1.1 Train Data

In [ ]:
# data load
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
train

In [ ]:
# info ccheck
display(train.info())
## non-null

# stat check
display(train.describe())

# unique value check
display(train.nunique())

---------------------------------
**Value**

x = [0,1,2]

y = [0,1,2,3]

congestion = [0 ~ 100]

direction = 8 direct

------------------------------------

In [ ]:
# "row_id" drop
train = train.drop('row_id',axis = 1)
train

In [ ]:
# dist check
sns.distplot(train["congestion"],
             hist = True, kde = False, color = 'green')
plt.show()

In [ ]:
# x 'variable' check
display(train.x.value_counts())

# y 'variable' check
display(train.y.value_counts())

# direction 'variable' check
display(train.direction.value_counts())

# congestion 'variable' check
display(train.congestion.value_counts())

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(x="x", y="congestion", data=train)
plt.title("x / congestion")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(x="y", y="congestion", data=train)
plt.title("y / congestion")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(x="direction", y="congestion", data=train)
plt.title("direction / congestion")
plt.show()

In [ ]:
import datetime

# "time" variable use

train.time = train.time.apply(pd.to_datetime)
train["month"] = train.time.dt.month
train["time_h"] = train.time.dt.hour
train["weekend"] = train.time.dt.weekday
train

In [ ]:
plt.figure(figsize=(10,10))


sns.barplot(x="month", y="congestion", data=train)
plt.title("month / congestion")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))


sns.barplot(x="time_h", y="congestion", data=train)
plt.title("time_h / congestion")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))


sns.barplot(x="weekend", y="congestion", data=train)
plt.title("weekend / congestion")
plt.show()

## 1.2 Test Data Check

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv')
test

In [ ]:
# info ccheck
display(test.info())
# stat check
display(test.describe())
# unique value check
display(test.nunique())

In [ ]:
# x 'variable' check
display(test.x.value_counts())

# y 'variable' check
display(test.y.value_counts())

# direction 'variable' check
display(test.direction.value_counts())


In [ ]:
test = test.drop('row_id',axis = 1)
test

In [ ]:
# "time" variable use
test.time = test.time.apply(pd.to_datetime)
test["month"] = test.time.apply(lambda x : x.month)
test["time_h"] = test.time.apply(lambda x : x.hour)
test["weekend"] = test.time.dt.weekday
test

# 2. Data PreProcssing

In [ ]:
display(train)

display(test)

In [ ]:
# "time" drop

for df in [train,test]:
    df.drop(['time','month'], axis=1, inplace=True)
    display(df)

In [ ]:
# labelencoding

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train.direction = le.fit_transform(train.direction)
test.direction = le.transform(test.direction)

display(train)
display(test)

In [ ]:
# Scaler

from sklearn.preprocessing import MinMaxScaler


mm = MinMaxScaler()
train[test.columns] = mm.fit_transform(train[test.columns])
test[test.columns] = mm.transform(test[test.columns])

display(train)
display(test)

# 3. Model

In [ ]:
# data 

X = train.drop("congestion", axis = 1)
y = train.congestion
display(X)
display(y)
display(test)

In [ ]:
# data split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, random_state = 42)

## 3.1 RandomForest

In [ ]:
# modelling

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

para = {'max_depth' : [2,5,10,20,50], #max_depth important
           'min_samples_leaf' : [2,4,8,16,32],
           'min_samples_split' : [2,4,8,16,32]}

# RF model
rf = RandomForestRegressor(random_state = 42)
# gridsearch
gs = GridSearchCV(rf, param_grid = para, cv = 3)
#train
gs.fit(X_train,y_train)
# best model estimate
gs_best = gs.best_estimator_

# display(gs.best_score_)
# best hyper_para
display(gs.best_params_)

In [ ]:
# evaluation
from sklearn.metrics import mean_squared_error

pred_train = gs_best.predict(X_train)

train_score = mean_squared_error(y_train,pred_train)

display(train_score)

In [ ]:
# evaluation
pred_test = gs_best.predict(X_test)

test_score = mean_squared_error(y_test,pred_test)

display(test_score)

In [ ]:
display(gs_best.feature_importances_)
display(X_train.columns)

df_import = pd.DataFrame({"col_name" : X_train.columns, "importance" : gs_best.feature_importances_})
df_import.sort_values(by = "importance")

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')

In [ ]:
pred = gs_best.predict(test)

sub.loc[:,"congestion"] = pred

display(sub)

sub.to_csv("submission_RF.csv",index = False)

## 3.2 LGBM

In [ ]:
# modelling

from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

para = {'max_depth' : [1, 5, 10, 20], 
           'n_estimators' : [1, 10, 50],
           'learning_rate' : [0.1, 0.2, 0.5]}

# LGBM model
lgbm = LGBMRegressor(random_state = 42)
#grid search
gs_lgbm = GridSearchCV(lgbm, param_grid = para, cv = 3)
# train
gs_lgbm.fit(X_train,y_train)
# best model estimate
gs_best_lgbm = gs_lgbm.best_estimator_

# display(gs.best_score_)
# best hyper_para
display(gs_lgbm.best_params_)

In [ ]:
# evaluation
from sklearn.metrics import mean_squared_error

pred_train = gs_best_lgbm.predict(X_train)

train_score = mean_squared_error(y_train,pred_train)

display(train_score)

In [ ]:
# evaluation
pred_test = gs_best_lgbm.predict(X_test)

test_score = mean_squared_error(y_test,pred_test)

display(test_score)

In [ ]:
display(gs_best_lgbm.feature_importances_)
display(X_train.columns)

df_import = pd.DataFrame({"col_name" : X_train.columns, "importance" : gs_best_lgbm.feature_importances_})
df_import.sort_values(by = "importance")

In [ ]:
pred = gs_best_lgbm.predict(test)

sub.loc[:,"congestion"] = pred

display(sub)

sub.to_csv("submission_LGBM.csv",index = False)

## 3.3 XGBoost

In [ ]:
import xgboost as xgb 

# xgboost parmeter 
para = {
    'min_child_weight': [1,3, 5,10],
    'gamma': [0.5, 1, 2, 5],
    'colsample_bytree': [0.6, 1.0],
    'max_depth': [3, 5, 10]}

# GridSearchCV
gs_XGB = GridSearchCV(xgb.XGBRegressor(random_state=42),
    param_grid=para, cv = 3) 
#train
gs_XGB.fit(X_train,y_train)
# best model estimate
gs_best_XGB = gs_XGB.best_estimator_

# display(gs.best_score_)
# best hyper_para
display(gs_XGB.best_params_)

In [ ]:
# evaluation
from sklearn.metrics import mean_squared_error

pred_train = gs_best_XGB.predict(X_train)

train_score = mean_squared_error(y_train,pred_train)

display(train_score)

In [ ]:
# evaluation
pred_test = gs_best_XGB.predict(X_test)

test_score = mean_squared_error(y_test,pred_test)

display(test_score)

In [ ]:
display(gs_best_XGB.feature_importances_)
display(X_train.columns)

df_import = pd.DataFrame({"col_name" : X_train.columns, "importance" : gs_best_XGB.feature_importances_})
df_import.sort_values(by = "importance")

In [ ]:
pred = gs_best_XGB.predict(test)

sub.loc[:,"congestion"] = pred

display(sub)

sub.to_csv("submission_XGB.csv",index = False)